# The biharmonic equation on the Torus

The biharmonic equation is given as

$$
\nabla^4 u = f.
$$

In this notebook we will solve this equation inside a torus, using curvilinear coordinates. 

<img src="https://cdn.jsdelivr.net/gh/spectralDNS/spectralutilities@master/figures/torus2.png">

The torus is parametrized by    

\begin{align*}
x(r, \theta, \phi) &= (R + r \cos \theta) \cos \phi \\
y(r, \theta, \phi) &= (R + r \cos \theta) \sin \phi \\
z(r, \theta, \phi) &= r \sin \theta
\end{align*}

where $(r, \theta, \phi) \in \Omega = [0, 1] \times [0, 2\pi] \times [0, 2\pi]$. Hence $\theta$ and $\phi$ are angles which make a full circle, so that their values start and end at the same point, $R$ is the distance from the center of the tube to the center of the torus,
$r$ is the radius of the tube. Note that $\theta$ is the angle in the small circle (around its center), whereas $\phi$ is the angle of the large circle, around origo.

We start the implementation by importing necessary functionality from shenfun and sympy and then defining the coordinates of the surface of the torus. Note that `rv` represents the position vector.
    

In [1]:
from shenfun import *
from shenfun.la import SolverGeneric2ND
import sympy as sp
from IPython.display import Math

N = 20
R = 3
r, theta, phi = psi = sp.symbols('x,y,z', real=True, positive=True)
rv = ((R + r*sp.cos(theta))*sp.cos(phi), (R + r*sp.cos(theta))*sp.sin(phi), r*sp.sin(theta)) 

B0 = Basis(N, 'L', bc='Biharmonic', domain=(0, 1))
B1 = Basis(N, 'F', dtype='D', domain=(0, 2*np.pi))
B2 = Basis(N, 'F', dtype='d', domain=(0, 2*np.pi))
T = TensorProductSpace(comm, (B0, B1, B2), coordinates=(psi, rv, sp.Q.positive(r*sp.cos(theta)+R)))
T.hi

array([1, x, x*cos(y) + 3], dtype=object)

Note that `T.hi` now contains the 3 scaling factors for the torus coordinates:

\begin{align*}
h_r &= \left|\frac{\partial \vec{r}}{\partial r}\right| = 1\\
h_{\theta} &= \left|\frac{\partial \vec{r}}{\partial \theta}\right| = r\\
h_{\phi} &= \left|\frac{\partial \vec{r}}{\partial \phi}\right| = r\cos \theta + R\\
\end{align*}

The covariant basis vectors used by shenfun are

In [2]:
Math(T.coors.latex_basis_vectors(covariant=True, symbol_names={r: 'r', theta: '\\theta', phi: '\\phi'}))

<IPython.core.display.Math object>

Now check what the biharmonic operator looks like for the torus

In [3]:
u = TrialFunction(T)
v = TestFunction(T)
du = div(grad(div(grad(u))))
Math((du).tolatex(symbol_names={r: 'r', theta: '\\theta', phi: '\\phi'}))

<IPython.core.display.Math object>

Glad you're not doing this by hand? 

To solve this equation we need to get a variational form that is separable. To get a variational form we multiply the equation by a weight $\omega$ and the complex conjugate of a test function $\overline{v}$, and integrate over the domain

\begin{align*}
\int_{\Omega} \nabla^4 u\, \overline{v} \omega dV &= \int_{\Omega} f \, \overline{v} \omega dV \\ 
\int_{\Omega} \nabla^4 u \, \overline{v} \omega r (r\cos \theta + R) dr d\theta d\phi &= \int_{\Omega} f \, \overline{v} \omega r (r\cos \theta + R) dr d\theta d\phi
\end{align*}

For Legendre and Fourier test functions the weight $\omega$ is normally a constant. However, we see that the denominator in some terms above contains $(r \cos \theta+3)^4$. This makes the variational form above inseparable. If, on the other hand, we change the weight $\omega$ to $(r \cos \theta+3)^3$, then the denominator disappears and the variational form becomes separable. 

$$
\int_{\Omega} \nabla^4 u \, \overline{v} \, r \,(r\cos \theta + R)^4 dr d\theta d\phi = \int_{\Omega} f \, \overline{v} \, r \,(r\cos \theta + R)^4 dr d\theta d\phi
$$

Below we achieve this by using `T.hi[2]**3`, where `T.hi[2]` = $h_{\phi} = (r\cos \theta + 3)$. We verify the implementation by using a manufactured solution that satisfies the boundary conditions. Note that a proper implementation should only consider a boundary condition on the surface of the torus, and not in the center. See, e.g., how this can be done for polar coordinates [here](https://shenfun.readthedocs.io/en/latest/polarhelmholtz.html).

In [4]:
ue = sp.sin(theta*2)*sp.cos(4*phi)*(1-r)*r*sp.sin(sp.pi*r)
f = div(grad(div(grad(u)))).tosympy(basis=ue, psi=psi)
fj = Array(T, buffer=f*T.hi[2]**3)
f_hat = Function(T)
f_hat = inner(v, fj, output_array=f_hat)
M = inner(v, div(grad(div(grad(u))))*T.hi[2]**3)
u_hat = Function(T)
sol = SolverGeneric2ND(M)
u_hat = sol(f_hat, u_hat)
uj = u_hat.backward()
uq = Array(T, buffer=ue)
print('Error =', np.linalg.norm(uj-uq))

Error = 5.211199484705211e-15


Note that the variational form contains

In [5]:
len(M)

62

tensorproduct matrices.

Finally, we look at the vector Laplacian and verify the following

$$
\nabla^2 \vec{u} = \nabla \nabla \cdot \vec{u} - \nabla \times \nabla \times \vec{u}
$$

The vector Laplace $\nabla^2 \vec{u}$ looks like: 

In [6]:
V = VectorTensorProductSpace(T)
p = TrialFunction(V)
du = div(grad(p))
Math(du.tolatex(symbol_names={r: 'r', theta: '\\theta', phi: '\\phi'}))

<IPython.core.display.Math object>

And if we subtract $\nabla \nabla \cdot \vec{u} - \nabla \times \nabla \times \vec{u}$ we should get the zero vector.

In [7]:
dv = grad(div(p)) - curl(curl(p))
dw = du-dv
dw.simplify()
Math(dw.tolatex(symbol_names={r: 'r', theta: '\\theta', phi: '\\phi'}))

<IPython.core.display.Math object>